In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "AdityaSingh312/Llama-7b-lamini-docs"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: C:\Users\adity\miniconda3\envs\tfgpu\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary C:\Users\adity\miniconda3\envs\tfgpu\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

C:\Users\adity\miniconda3\envs\tfgpu\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adity\.cache\huggingface\hub\models--AdityaSingh312--Llama-7b-lamini-docs. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
import logging
logger = logging.getLogger(__name__)
global_config = None

device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

In [4]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=150):
  # Tokenize
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=150)
    generated_text_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
  # Strip the prompt

    return generated_text_answer

In [6]:
test_text = "Can you tell me something about Lamini?"
print("Question input (test):", test_text)
#print(f"Correct answer from Lamini docs: {test_data[0]['answer']}")
print("Model's answer: ")
print(inference(test_text, model, tokenizer))

Question input (test): Can you tell me something about Lamini?
Model's answer: 
Can you tell me something about Lamini?Lamini is an AI library that allows you to create and train your own LLMs. It is written in Python and can be used with a variety of datasets and models. You can use Lamini to train your own LLMs for natural language processing tasks such as text classification, sentiment analysis, and machine translation. Additionally, Lamini provides a simple interface for fine-tuning pre-trained models on your own data. This allows you to customize the LLM to your specific use case and improve its performance. For more information, see the Lamini documentation at https://lamini-ai.github.io/.


In [7]:
import gradio as gr

In [11]:
def answer(prompt):
    ans = inference(prompt, model, tokenizer=tokenizer, max_input_tokens=1000, max_output_tokens=150)
    return ans

In [12]:
ui = gr.Interface(answer, inputs="text", outputs="text")
ui.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://284adc7ffd7c6a8247.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
